In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import requests
import json

# Scrap all MotoGP standings from all seasons

In [ ]:
# Scrap all seasons IDs
response = requests.get("https://api.motogp.pulselive.com/motogp/v1/results/seasons")

## Check if the request was successful
if response.status_code == 200:
    data = response.json()  # Parse the JSON response directly

    seasons_df = pd.DataFrame(data)  # Create a DataFrame from the JSON data
    seasons_df = seasons_df[["id", "year"]]
    seasons_ids = seasons_df["id"].tolist()

    print()
    print("Seasons successfully processed:")
    print(f"This is the list: {seasons_ids}")
else:
    print("Failed to retrieve the page.")

# Scrap MotoGP ID
response = requests.get(
    "https://api.motogp.pulselive.com/motogp/v1/results/categories?seasonUuid=db8dc197-c7b2-4c1b-b3a4-7dc723c087ed"
)

## Check if the request was successful
if response.status_code == 200:
    try:
        json_data = response.json()
        print()
        print("JSON data retrieved successfully")

        # Find MotoGP category and extract its ID
        for category in json_data:
            if category["name"] == "MotoGP™":
                motogp_id = category["id"]
                print(f"MotoGP id = {motogp_id}")
                break
    except json.JSONDecodeError:
        print()
        print("Failed to parse JSON response.")
else:
    print()
    print("Failed to retrieve the page.")

# Scrap all MotoGP seasons championships
print()
# Initialize an empty list to hold all data
all_data = []

# Loop through seasons IDs and retrieve MotoGP standings
count = 0
for season_id in seasons_ids:
    count = count + 1
    url = f"https://api.motogp.pulselive.com/motogp/v1/results/standings?seasonUuid={season_id}&categoryUuid={motogp_id}"
    response = requests.get(url)

    print(
        f"Season ID: {season_id}, Status Code: {response.status_code}, Count = {count}"
    )

    if response.status_code == 200:
        json_data = response.json()

        # Extract relevant information from the JSON response
        data = []
        for entry in json_data.get("classification", []):
            rank = entry.get("position")
            rider_name = entry.get("rider", {}).get("full_name")
            country = entry.get("rider", {}).get("country", {}).get("iso")

            # Check if the "team" object exists before accessing its attributes
            team_info = entry.get("team")
            team_name = team_info.get("name") if team_info else "No Team"

            year = None
            if team_info and "season" in team_info:
                year = entry.get("team", {}).get("season", {}).get("year")

            constructor = entry.get("constructor", {}).get("name")

            data.append([rank, rider_name, country, team_name, constructor, year])

        # Append the data for the current season to the all_data list
        all_data.extend(data)

    else:
        print("Failed to retrieve standings for season", season_id)

# Create a DataFrame from the collected data
columns = ["Rank", "Rider Name", "Country", "Team Name", "Constructor", "Year"]
global_df = pd.DataFrame(all_data, columns=columns)

# Print the global DataFrame or do whatever you need with it
print()
print(global_df)

global_df.to_parquet("extracted_datas.parquet", index=False)

# Scrap MotoGP race results from all seasons 

In [ ]:
# Define the API URLs
seasons_url = "https://api.motogp.pulselive.com/motogp/v1/results/seasons"
events_url = "https://api.motogp.pulselive.com/motogp/v1/results/events"
results_url = "https://api.motogp.pulselive.com/motogp/v1/results/session/e38536d9-bf81-4613-b0bf-91ec5ea13de1/classification?test=false"

# Fetch data from the API
seasons_response = requests.get(seasons_url)
seasons_data = seasons_response.json()

# Create an empty list to store data
results_data_list = []

# Loop through each season
for season in seasons_data:
    season_year = season["year"]

    # Fetch events data for the current season
    events_params = {"seasonUuid": season["id"], "isFinished": True}
    events_response = requests.get(events_url, params=events_params)
    events_data = events_response.json()

    # Loop through each event (race) in the current season
    for event in events_data:
        circuit_name = event["circuit"]["name"]

        # Fetch results data for the current event
        results_response = requests.get(results_url)
        results_data = results_response.json()["classification"]

        # Loop through each result in the current event
        for result in results_data:
            rank = result["position"]
            rider_name = result["rider"]["full_name"]
            nationality = result["rider"]["country"]["name"]
            team_name = result["team"]["name"]
            constructor_name = result["constructor"]["name"]

            # Append the data to the list
            results_data_list.append(
                {
                    "Season": season_year,
                    "Circuit": circuit_name,
                    "Rank": rank,
                    "Rider": rider_name,
                    "Nationality": nationality,
                    "Team": team_name,
                    "Constructor": constructor_name,
                }
            )

# Create a DataFrame from the list of data
results_df = pd.DataFrame(results_data_list)

# Display the resulting DataFrame
print(results_df)

# Connect to PostgreSQL

In [ ]:
# Replace these values with your PostgreSQL database connection details
db_params = {
    "host": "127.0.0.1",
    "dbname": "motogp_db",
    "user": "alasnier",
    "password": "alasnier",
    "port": 3030,
}

# Create a connection to the database
conn = psycopg2.connect(**db_params)

# Assuming your DataFrame is called 'results_df'
table_name = "motogp_table"

# Create a SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
)

# Export the DataFrame to PostgreSQL
results_df.to_sql(table_name, engine, if_exists="replace", index=False)

In [ ]:
conn.close()